## **Perkenalan Dataset**


##### Variabel-variabel pada Diabetes prediction dataset adalah sebagai berikut:
- **`gender`** :Jenis kelamin yang merujuk pada perbedaan biologis antara laki-laki dan perempuan, yang dapat memengaruhi risiko seseorang terhadap diabetes. Terdapat tiga kategori pada variabel ini: laki-laki, perempuan, dan lainnya.
- **`age`** : Usia merupakan faktor yang sangat penting karena diabetes lebih sering didiagnosis pada individu yang lebih tua. Dalam dataset ini, rentang usia yang tercatat adalah antara 0 hingga 80 tahun.
- **`hypertension`** : Hipertensi adalah kondisi medis di mana tekanan darah tinggi dapat meningkatkan risiko penyakit, termasuk diabetes. Nilainya adalah 0 atau 1, di mana 0 berarti tidak memiliki hipertensi dan 1 berarti menderita hipertensi.
- **`heart_disease`** : Penyakit jantung merupakan faktor risiko lain yang berkontribusi pada peningkatan kemungkinan seseorang mengidap diabetes. Nilai pada variabel ini adalah 0 atau 1, dengan 0 menunjukkan tidak ada penyakit jantung dan 1 berarti pasien memiliki penyakit jantung.
- **`smoking_history`** : Riwayat merokok berperan sebagai faktor tambahan dalam peningkatan risiko diabetes, serta memperburuk dampak komplikasi. Terdapat 5 kategori yang tersedia dalam data: tidak merokok saat ini, merokok sebelumnya, tidak ada informasi, merokok saat ini, dan merokok pernah.
- **`bmi`** : Indeks Massa Tubuh (BMI) mengukur komposisi tubuh berdasarkan berat dan tinggi badan. BMI yang tinggi sering dikaitkan dengan peningkatan risiko diabetes. Rentang BMI dalam dataset ini adalah dari 10,16 hingga 71,55. Kategori BMI adalah: kurang dari 18,5 (kekurangan berat badan), 18,5 hingga 24,9 (normal), 25 hingga 29,9 (kelebihan berat badan), dan 30 atau lebih (obesitas).
- **`HbA1c_level`** : Kadar HbA1c (Hemoglobin A1c) mengukur rata-rata kadar gula darah dalam dua hingga tiga bulan terakhir. Kadar HbA1c yang lebih tinggi menunjukkan risiko lebih besar untuk diabetes. Biasanya, kadar lebih dari 6,5% menandakan adanya diabetes.
- **`blood_glucose_level`** : Kadar glukosa darah mengacu pada jumlah glukosa yang ada dalam darah pada suatu waktu tertentu. Kadar glukosa darah yang tinggi merupakan indikator utama diabetes.
- **`diabetes`** : Variabel target yang akan diprediksi, dengan nilai 1 menunjukkan pasien mengidap diabetes dan 0 berarti tidak mengidap diabetes.

## **1. Import Library**

Mengimport library-library yang dibutuhkan untuk membuat model

In [32]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns 

from sklearn.preprocessing import OneHotEncoder, StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 

## **2. Data Loading**

Load dataset dan melihat 5 dataset teratas

In [ ]:
data = pd.read_csv(r'C:\KULIAH STATISTIKA UNIVERSITAS SYIAH KUALA\SEMESTER 6\Coding Camp by DBS\Submission Project Akhir Kelas\Machine Learning Terapan\Submission\Dataset.csv')
data.head()

: 

## **3. Eksplorasi Data**

### 3.1. Deskripsi Variabel

Malihat struktur dataset dan indikasi adanya missing value.

In [ ]:
data.info()

: 

**Insight**
- Tidak ada indikasi missing value

Analisis statistik deskriptif pada variabel

In [ ]:
data.describe()

: 

### 3.2. Menangani Missing Value dan Outlier

Melihat apakah terdapat missing values pada dataset

In [ ]:
data.isnull().sum()

: 

**Insight**
- Tidak ada missing value

Melihat total kategori pada fitur gender

In [ ]:
print(data['gender'].value_counts())

: 

**Insight**
- Terdapat kategori yang hanya ada 18 dari total keseluruhan data dan sangat timpang jumlah data keseluruhan.

Mangganti kategori yang jumlahnya sangat kecil dengan modus dari kategori lain.

In [ ]:
gender = data['gender'].mode()[0]
data['gender'] = data['gender'].replace('Other', gender)

: 

Mengecek dan menangani Outlier

In [ ]:
numeric = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
outlierValues = {}
data_before_clipping = data[numeric].copy()

# Deteksi dan Clipping Outlier
for col in numeric:
    q1, q3 = np.percentile(data[col].dropna(), [25, 75])
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr

    outliers = data[col][(data[col] < lower) | (data[col] > upper)]
    outlierValues[col] = outliers

    data[col] = np.clip(data[col], lower, upper)

# Visualisasi Boxplot SEBELUM Clipping
fig, axes = plt.subplots(1, len(numeric), figsize=(5 * len(numeric), 5))
fig.suptitle("Boxplot Sebelum Clipping", fontsize=18, fontweight='bold')
for ax, col in zip(axes, numeric):
    sns.boxplot(x=data_before_clipping[col], ax=ax, color='salmon')
    ax.set_title(col)
plt.tight_layout(rect=[0, 0, 1, 0.95]) 
plt.show()

# Visualisasi Boxplot SESUDAH Clipping
fig, axes = plt.subplots(1, len(numeric), figsize=(5 * len(numeric), 5))
fig.suptitle("Boxplot Sesudah Clipping", fontsize=18, fontweight='bold')
for ax, col in zip(axes, numeric):
    sns.boxplot(x=data[col], ax=ax, color='mediumseagreen')
    ax.set_title(col)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


: 

### 3.3. Univariate Analysis

Mengelompokan kolom ke numerik dan kategorikal.

In [ ]:
numeric = ['age','bmi','HbA1c_level','blood_glucose_level']
categoric = ['gender', 'hypertension', 'heart_disease', 'smoking_history', 'diabetes']

: 

Visualisasi distibusi nilai kolom numerik.

In [ ]:
plt.figure(figsize=(5 * len(numeric), 4))

colors = sns.color_palette("Set2", len(numeric))

for i, col in enumerate(numeric):
    plt.subplot(1, len(numeric), i + 1)
    
    if data[col].nunique() > 10:
        sns.histplot(data[col].dropna(), bins=30, kde=True, color=colors[i])
    else:
        sns.countplot(x=data[col], color=colors[i])

    plt.title(f"Distribusi {col}")
    plt.xlabel(col)
    plt.ylabel("Frekuensi")

plt.tight_layout()
plt.show()

: 

Visualisasi distibusi nilai kolom Kategori.

In [ ]:
plt.figure(figsize=(15, 12))

for i, col in enumerate(categoric[:6], 1):  # Ambil maksimal 6 kolom pertama
    plt.subplot(3, 2, i)
    unique_vals = data[col].nunique()
    palette = sns.color_palette("Set3", unique_vals)
    
    ax = sns.countplot(
        x=data[col],
        hue=data[col],
        order=data[col].value_counts().index,
        palette=palette,
        legend=False,
        edgecolor="black"
    )
    
    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2., height),
                    ha='center', va='bottom', fontsize=10)

    plt.title(f"Distribusi {col}")
    plt.ylabel("Frekuensi")
    plt.xlabel(col)
    plt.xticks(rotation=0)

plt.tight_layout()
plt.show()


: 

### 3.4. Multivariate Analysis

Melihat rata rata diabetes dibandingakn fitur lain dalam dataset

In [ ]:
categoric = ['gender', 'hypertension', 'heart_disease', 'smoking_history']

fig, axes = plt.subplots(1, len(categoric), figsize=(20, 5))

for i, col in enumerate(categoric):
    sns.barplot(
        x=col,
        y="diabetes",
        data=data,
        hue=col,
        palette="pastel",
        estimator=np.mean,
        legend=False,
        edgecolor="black",
        ax=axes[i]
    )
    axes[i].set_title(f"Rata-rata diabetes per {col}")
    axes[i].set_ylabel("Rata-rata")
    axes[i].set_xlabel(col)

plt.tight_layout()
plt.show()


: 

Visualisasikan hubungan antar variabel

In [ ]:
sns.pairplot(data, diag_kind='kde', kind='scatter', plot_kws={'alpha':0.6, 's':40}, corner=True)
plt.show()

: 

Visualisasi correlation matrix

In [ ]:
numeric = data.select_dtypes(include=['number']).columns

plt.figure(figsize=(10, 8))
correlation_matrix = data[numeric].corr().round(2)

sns.heatmap(
    correlation_matrix, 
    annot=True, 
    cmap='viridis',
    linewidths=0.5, 
    fmt='.2f', 
    cbar_kws={"shrink": 0.8}
)

plt.title("Correlation Matrix", fontsize=20)
plt.xticks(rotation=45, fontweight='bold')
plt.yticks(rotation=0, fontweight='bold')
plt.tight_layout()
plt.show()


: 

## **4. Data Preparation**

### 4.1. Encoding Fitur Kategori

Encoding Variabel gender dan smoking_history menggunakan OneHotEncoder serta cek hasil dari encoding.

In [ ]:
categoric = ['gender', 'smoking_history']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

encoded_data = pd.DataFrame(
    encoder.fit_transform(data[categoric]),
    columns=encoder.get_feature_names_out(categoric),
    index=data.index
)

data = data.drop(columns=categoric).join(encoded_data)
data.head()

: 

### 4.2. Standarisasi Fitur

Melakukan standarisasi menggunakan StandardScaler untuk menyamakan rentang nilai dan melihat hasil perubahan

In [ ]:
numeric = ['age', 'bmi', 'blood_glucose_level', 'HbA1c_level']
scaler = StandardScaler()
data[numeric] = scaler.fit_transform(data[numeric])
data.head()

: 

### 4.3. Spliting Data

Membagi data menjadi train dan test dengan komposisi 20 % untuk test dan 80% untuk train

In [ ]:
X = data.drop(columns=["diabetes"])  
y = data["diabetes"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

: 

## **5. Model Development**

### 5.1. Random Forest

Load model Random Forest

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

: 

### 5.2. Naive Bayes

Load model Naive Bayes

In [ ]:
naive_bayes = BernoulliNB()
naive_bayes.fit(X_train, y_train)

: 

### 5.3 Decision Tree

Load model Decision Tree

In [ ]:
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

: 

## **6. Evaluasi Model**

### 6.1. Random Forest

Evaluasi metrik accuracy_score dan classification_report (precision, recall dan f1-score) pada model Random Forest.

In [ ]:
y_pred_rf = random_forest.predict(X_test)

print(f"Akurasi: {accuracy_score(y_test, y_pred_rf):.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

: 

### 6.2. Naive Bayes

Evaluasi model Naive Bayes.

In [ ]:
y_pred_nb = naive_bayes.predict(X_test)

print(f"Akurasi: {accuracy_score(y_test, y_pred_nb):.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred_nb))

# Confusion Matrix
cm_nb = confusion_matrix(y_test, y_pred_nb)
sns.heatmap(cm_nb, annot=True, fmt='d', cmap='Reds')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

: 

### 6.3. Decision Tree

Evaluasi model Decision Tree.

In [ ]:
y_pred_dt = decision_tree.predict(X_test)

print(f"Akurasi: {accuracy_score(y_test, y_pred_dt):.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred_dt))

# Confusion Matrix
cm_dt = confusion_matrix(y_test, y_pred_dt)
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

: 